# Extending the Generalized Sector

Table of Contents:

1. [Extending outputs](#Extending-outputs)
    1. [Simple output extension](#Simple-output-extension)
    1. [Adding TOML parameters to the outputs](#Adding-TOML-parameters-to-the-outputs)
1. [Modeling early obsolescence](#Modeling-early-obsolescence)
1. [Where to store new functionality](#Where-to-store-new-functionality)

One key feature of the generalized sector's implementation is that it should be easy to
extend. As such, most the steps shown in [Generalized Sector
Overview](GeneralizedSectorOverview.ipynb) can be made to run custom python functions,
as long as these inputs and output of the function follow a standard specific to each
step. We will look at a few here. Below is a list of possible hooks, referenced by their
implementation in the MUSE model itself

- `register_interaction_net` in `muse.interactions`: a list of lists of agents
  that do interact together.
- `register_agent_interaction` in `muse.interactions`: Given a list of
  interacting agents, perform the interaction itself.
- `register_production` in `muse.production`: A method to compute the production
  from a sector, given the demand and the capacity.
- `register_initial_asset_transform` in `muse.hooks`: Prior to investing,
  within an agent, allows any kind of transformation to be applied to the assets.
- `register_final_asset_transform` in `muse.hooks`: After computing investment,
  within an agent, sets the assets that will be owned the agents.
- `register_demand_share` in `muse.demand_share`: During agent investment, share
  of the demand that an agent will try and satisfy.
- `register_filter` in `muse.filters`: During agent investment, a
  filter to remove technologies from consideration.
- `register_objective` in `muse.objectives`: During agent investment, a
  quantity defining preference between one or another technology during investment.
- `register_decision` in `muse.decisions`: During agent investment, a transform
  to aggregate multiple objectives into a single one, e.g. via a weighted sum.
- `register_investment` in `muse.investment`: During agent investment, matches
  the demand to future investment using the decision metric above.
- `register_output_quantity` in `muse.output.sector`: A sectorial quantity to output for
  postmortem analysis.
- `register_output_sink` in `muse.outputs`: A _place_ where to store an output
  quantity, e.g. a file with a given format, a database on premise or on the cloud,
  etc...
- `register_carbon_budget_fitter` in `muse.carbon_budget`
- `register_carbon_budget_method` in `muse.carbon_budget`
- `register_sector`: registers a function that can create a sector from a muse
  configuration object.

## Extending outputs

MUSE can be told to save quantities for post-mortem analysis. There are actually two
steps to this process. First, we want to compute the quantity of interest. Second, we
want to store in a __sink__, somewhere. Generally, we can mix and match between
quantities and sinks; for instance we can compute the capacity and save it as csv file,
or an netcdf file.

### Simple output extension

To demonstrate this, we will compute a new quantity of (little) interest, and then save
it as is to a text file:

In [1]:
from xarray import Dataset, DataArray
from muse import outputs

@outputs.sector.register_output_quantity
def bracket(
    market: Dataset,
    capacity: DataArray,
    technologies: Dataset
) -> DataArray:
    from numpy import logical_and
    return logical_and(capacity > 10, capacity < 100)

The function we wrote takes three arguments. These are mandatory **for this hook**, the
hook which allows compute quantities for post-mortem analysis. We will see shortly other
hooks require other arguments. The function outputs the quantity that we want to save.

Importantly, the function is _decorated_: on the line above it sits the decorator
`@register_output_quantity`. This decorator ensures the new quantity will be addressable
in the TOML file.

Now we can write a sink. In this case, the sink will simply dump the quantity it is
given to a file, with a small message:

In [2]:
from typing import Any, Text
from muse.outputs.sinks import register_output_sink, sink_to_file

@register_output_sink(name="txt")
@sink_to_file(".txt")
def text_dump(data: Any, filename: Text) -> None:
    from pathlib import Path
    Path(filename).write_text(f"Hello, world!\n\n{data}")

There code above makes use of two decorators. The first one we expect,
`@register_output_sink`. It registers the function with MUSE, so that the sink is
addressable from a TOML file. The second one, `@sink_to_file`, is optional. It adds to
sinks that are files some nice-to-have features, such as a way to specify filenames and
check that files cannot be overwritten, unless explicitly allowed to.

Now we want to modify the input file to actually use this output type. Specifically, we
add a section to the output table:


```toml
[[sectors.commercial.outputs]]
quantity = "bracket"
sink = "txt"
filename = "{cwd}/{default_output_dir}/{Sector}{Quantity}{year}{suffix}"
```


The last line above allows us to specify the name of the file. We could also use
`sector` (no caps) and `quantity`.

There can be as many sections of this kind as we want in the TOML file, allowing for
multiple outputs.

In the following, we  first copy the default model provided with muse to a local subfolder called "model". Then we read the `settings.toml` file and modify it using python. You may prefer to modify the `settings.toml` file using your favorite text editor. However, modifying the file programmatically allows us to
routinely run this notebook as part of MUSE's test suite and check that the tutorial it is still up
to date.

In [3]:
from pathlib import Path
from toml import load, dump
from muse import examples

model_path = examples.copy_model(overwrite=True)
settings = load(model_path / "settings.toml")
new_output = {
    "quantity": "bracket",
    "sink":  "txt",
    "overwrite": True,
    "filename": "{cwd}/{default_output_dir}/{Sector}{Quantity}{year}{suffix}",
}
settings["sectors"]["residential"]["outputs"].append(new_output)
dump(settings, (model_path / "modified_settings.toml").open("w"))
settings

{'time_framework': [2020, 2025, 2030, 2035, 2040, 2045, 2050],
 'foresight': 5,
 'regions': ['USA'],
 'interest_rate': 0.1,
 'interpolation_mode': 'Active',
 'log_level': 'info',
 'equilibrium_variable': 'demand',
 'maximum_iterations': 100,
 'tolerance': 0.1,
 'tolerance_unmet_demand': -0.1,
 'carbon_budget_control': {'budget': []},
 'global_input_files': {'projections': '{path}/input/Projections.csv',
  'global_commodities': '{path}/input/GlobalCommodities.csv'},
 'sectors': {'residential': {'type': 'default',
   'priority': 1,
   'investment_production': 'share',
   'dispatch_production': 'share',
   'agents': '{path}/technodata/Agents.csv',
   'technodata': '{path}/technodata/residential/Technodata.csv',
   'commodities_in': '{path}/technodata/residential/CommIn.csv',
   'commodities_out': '{path}/technodata/residential/CommOut.csv',
   'existing_capacity': '{path}/technodata/residential/Existing.csv',
   'outputs': [{'filename': '{cwd}/{default_output_dir}/{Sector}/{Quantity}/{yea

We can now try and run the simulation. There are two ways to do this. From the
command-line, we can do `python3 -m muse data/commercial/settings2.toml` (note
that slashes may be the other way on Windows). Directly from the notebook, we can do:

In [4]:
from muse.mca import MCA

mca = MCA.factory(model_path / "modified_settings.toml")
mca.run()

-- 2020-05-13 10:09:36 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for retrofit agent A1 of power sector in year 2020.

-- 2020-05-13 10:09:41 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for retrofit agent A1 of power sector in year 2020.

-- 2020-05-13 10:09:47 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for newcapa agent A1 of gas sector in year 2025.

-- 2020-05-13 10:09:47 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for retrofit agent A1 of gas sector in year 2025.

-- 2020-05-13 10:09:52 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for newcapa agent A1 of gas sector in year 2025.

-- 2020-05-13 10:09:52 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for retrofit agent A1 of gas sector in year 2025.

-- 2020-05-13 10:09:56 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for retrofit agent A1 of power sector in year 20

We can check that running the simulation has created the files we expect:

In [5]:
all_txt_files = sorted((Path() / "Results").glob("Residential*.txt"))
assert len(all_txt_files) == 7
assert "Hello, world!" in all_txt_files[0].read_text()
all_txt_files

[PosixPath('Results/ResidentialBracket2020.txt'),
 PosixPath('Results/ResidentialBracket2025.txt'),
 PosixPath('Results/ResidentialBracket2030.txt'),
 PosixPath('Results/ResidentialBracket2035.txt'),
 PosixPath('Results/ResidentialBracket2040.txt'),
 PosixPath('Results/ResidentialBracket2045.txt'),
 PosixPath('Results/ResidentialBracket2050.txt')]

### Adding TOML parameters to the outputs

The bracket quantity we are computing would be (somewhat) more useful if we could
specify the bracket range straight from the TOML file. Similarly, the "hello world"
message in the sink could also do with being parameterized. Not all hooks are this
flexible (for historical reasons, rather than any intrinsic difficulty). However, for
outputs, we can do this as follows:

In [6]:
from typing import Optional

@outputs.sector.register_output_quantity(overwrite=True)
def bracket(
    market: Dataset,
    capacity: DataArray,
    technologies: Dataset,
    lower: Optional[int] = 10,
    upper: Optional[int] = 100,
) -> DataArray:
    from numpy import logical_and
    return logical_and(capacity > lower, capacity < upper)


@register_output_sink(name="txt", overwrite=True)
@sink_to_file(".txt")
def text_dump(
    data: Any,
    filename: Text,
    msg : Optional[Text] = "Hello, world!"
) -> None:
    from pathlib import Path
    Path(filename).write_text(f"{msg}\n\n{data}")

We simply added parameters as keyword arguments to the functions.

Note: The `overwrite` argument allows us to overwrite previously defined registered
    functions. This is useful in a notebook such as this. But it should not be used in
    general. If `overwrite` were false, then the code would issue a warning and it would
    leave the TOML to refer to the original functions at the beginning of the notebook.
    This is useful to ensure custom modules do not silently overwrite each others registered functions.

Now we can modify the output section to take additional arguments:


```toml
[[sectors.commercial.outputs]]
quantity.name = "bracket"
quantity.upper = 10
quantity.lower = 1
sink.name = "txt"
sink.filename = "{cwd}/{default_output_dir}/{Sector}{Quantity}{year}{suffix}"
sink.msg = "Hello, you!"
sink.overwrite = True
```

Now, both `sink` and `quantity` are dictionaries which can take any number of arguments.
Previously, we were using a shorthand for convenience.

In [7]:
from toml import load, dump
from pathlib import Path

settings = load(model_path / "settings.toml")
settings["sectors"]["residential"]["outputs"] = [
    {
        "quantity": {
            "name": "bracket",
            "upper": 10,
            "lower": 1
        },
        "sink":  {
            "name": "txt", 
            "filename": "{cwd}/{default_output_dir}/{Sector}{Quantity}{year}{suffix}",
            "msg": "Hello, you!",
            "overwrite": True,
        }
    }
]
dump(settings, (model_path / "modified_settings.toml").open("w"))
settings

{'time_framework': [2020, 2025, 2030, 2035, 2040, 2045, 2050],
 'foresight': 5,
 'regions': ['USA'],
 'interest_rate': 0.1,
 'interpolation_mode': 'Active',
 'log_level': 'info',
 'equilibrium_variable': 'demand',
 'maximum_iterations': 100,
 'tolerance': 0.1,
 'tolerance_unmet_demand': -0.1,
 'carbon_budget_control': {'budget': []},
 'global_input_files': {'projections': '{path}/input/Projections.csv',
  'global_commodities': '{path}/input/GlobalCommodities.csv'},
 'sectors': {'residential': {'type': 'default',
   'priority': 1,
   'investment_production': 'share',
   'dispatch_production': 'share',
   'agents': '{path}/technodata/Agents.csv',
   'technodata': '{path}/technodata/residential/Technodata.csv',
   'commodities_in': '{path}/technodata/residential/CommIn.csv',
   'commodities_out': '{path}/technodata/residential/CommOut.csv',
   'existing_capacity': '{path}/technodata/residential/Existing.csv',
   'outputs': [{'quantity': {'name': 'bracket', 'upper': 10, 'lower': 1},
     '

We then run the simulation again:

In [8]:
import logging
from muse.mca import MCA
logging.getLogger("muse").setLevel(0)
mca = MCA.factory(model_path / "modified_settings.toml")
mca.run()

-- 2020-05-13 10:12:28 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for retrofit agent A1 of power sector in year 2020.

-- 2020-05-13 10:12:30 - muse.mca - WARNING
Check growth constraints for wind.

-- 2020-05-13 10:12:34 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for retrofit agent A1 of power sector in year 2020.

-- 2020-05-13 10:12:36 - muse.mca - WARNING
Check growth constraints for wind.

-- 2020-05-13 10:12:40 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for newcapa agent A1 of gas sector in year 2025.

-- 2020-05-13 10:12:40 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for retrofit agent A1 of gas sector in year 2025.

-- 2020-05-13 10:12:40 - muse.mca - WARNING
Check growth constraints for wind.

-- 2020-05-13 10:12:45 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for newcapa agent A1 of gas sector in year 2025.

-- 2020-05-13 10:12:45 - muse.sectors.secto

And we can check the parameters were used accordingly:

In [10]:
all_txt_files = sorted((Path() / "Results").glob("Residential*.txt"))
assert len(all_txt_files) == 7
assert "Hello, you!" in all_txt_files[0].read_text()
all_txt_files

[PosixPath('Results/ResidentialBracket2020.txt'),
 PosixPath('Results/ResidentialBracket2025.txt'),
 PosixPath('Results/ResidentialBracket2030.txt'),
 PosixPath('Results/ResidentialBracket2035.txt'),
 PosixPath('Results/ResidentialBracket2040.txt'),
 PosixPath('Results/ResidentialBracket2045.txt'),
 PosixPath('Results/ResidentialBracket2050.txt')]

## Modeling early obsolescence

The objective is to incite agents to replace their assets before they reach end of life.
To do this, we will create a depressed `production` function. The production will use
existing methods but depress the result via a factor that can be controlled by the user
from the toml:

In [11]:
from muse.production import register_production

@register_production(name="depressed shares")
def depressed_capacity_production(
    market: Dataset,
    capacity: DataArray,
    technologies: Dataset,
    factor: Optional[float] = 0.9,
    **kwargs,
) -> DataArray:
    """Comfort for input techs."""
    from muse.production import maximum_production
    from logging import getLogger

    getLogger("muse.production").critical(f"Production is depressed by {factor}")
    return factor * maximum_production(market, capacity, technologies, **kwargs)

The function itself is straightforward. It takes as input the market (`consumption`,
`supply`, `prices`) as received from the MCA, the `capacity` aggregated across the
sector, and the dataset describing the technologies. We as argument the factor by which
to artificially depress the production.

This function must hooked just in the right place. It should intervene only when
computing the demand against which agents invest. It should not be used when computing
the actual amount of commodities produced by a sector and returned to the MCA.

Fortunately, sectors have access to two separate hooks for this:



```TOML
[sectors.commercial]
supply = 'shares'
production = 'shares'
```



- `production` defines the method to compute the demand ahead of investing
- `supply` defines the method to compute the `consumption` and `supply` returned by the
    sector

In order to use our newly created functionality, we will modify the settings to read:


```TOML
[sectors.commercial]
supply = 'shares'
production.name = 'depressed shares'
production.factor = 0.9
```

In [12]:
settings = load(model_path / "settings.toml")
settings["sectors"]["residential"]["production"] = {
    "name": "depressed shares",
    "factor": 0.9,
}
dump(settings, (model_path / "modified_settings.toml").open("w"))
settings

{'time_framework': [2020, 2025, 2030, 2035, 2040, 2045, 2050],
 'foresight': 5,
 'regions': ['USA'],
 'interest_rate': 0.1,
 'interpolation_mode': 'Active',
 'log_level': 'info',
 'equilibrium_variable': 'demand',
 'maximum_iterations': 100,
 'tolerance': 0.1,
 'tolerance_unmet_demand': -0.1,
 'carbon_budget_control': {'budget': []},
 'global_input_files': {'projections': '{path}/input/Projections.csv',
  'global_commodities': '{path}/input/GlobalCommodities.csv'},
 'sectors': {'residential': {'type': 'default',
   'priority': 1,
   'investment_production': 'share',
   'dispatch_production': 'share',
   'agents': '{path}/technodata/Agents.csv',
   'technodata': '{path}/technodata/residential/Technodata.csv',
   'commodities_in': '{path}/technodata/residential/CommIn.csv',
   'commodities_out': '{path}/technodata/residential/CommOut.csv',
   'existing_capacity': '{path}/technodata/residential/Existing.csv',
   'outputs': [{'filename': '{cwd}/{default_output_dir}/{Sector}/{Quantity}/{yea

Finally, we can run the simulation and check in the log that our function did run:

In [13]:
mca = MCA.factory(model_path / "modified_settings.toml")
mca.run()

-- 2020-05-13 10:17:14 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for retrofit agent A1 of power sector in year 2020.

-- 2020-05-13 10:17:16 - muse.mca - WARNING
Check growth constraints for wind.

-- 2020-05-13 10:17:19 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for retrofit agent A1 of power sector in year 2020.

-- 2020-05-13 10:17:21 - muse.mca - WARNING
Check growth constraints for wind.

-- 2020-05-13 10:17:25 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for newcapa agent A1 of gas sector in year 2025.

-- 2020-05-13 10:17:25 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for retrofit agent A1 of gas sector in year 2025.

-- 2020-05-13 10:17:26 - muse.mca - WARNING
Check growth constraints for wind.

-- 2020-05-13 10:17:30 - muse.sectors.sector - CRITICAL
No demand, no investment needed for for newcapa agent A1 of gas sector in year 2025.

-- 2020-05-13 10:17:30 - muse.sectors.secto

Note: Another approach to model early obsolescence would be to modify the `demand_share`
    hooks which determine the demand againts which each agent invests.

## Where to store new functionality

As was demonstrated above, new functionality can be added easily. However, running a
jupyter notebook is not always the best approach. One can also store the functions in an
arbitrary python file:

In [14]:
%%writefile mynewfunctions.py
from muse.outputs.sinks import register_output_sink, sink_to_file

@register_output_sink(name="dummy")
@sink_to_file(".dummy")
def dummy(data, filename):
    from pathlib import Path
    Path(filename).write_text("Nothing to see")

Writing mynewfunctions.py


Then we can tell the TOML file where to find it:


```toml
plugins = "{cwd}/mynewfunctions.py"

[[sectors.commercial.outputs]]
quantity = "capacity"
sink = "dummy"
overwrite = true
```

Alternatively, `plugin` can also be given a list of paths rather than just a single one, as done below.

In [15]:
settings = load(model_path / "settings.toml")
settings["plugins"] = ["{cwd}/mynewfunctions.py"]
settings["sectors"]["residential"]["outputs"] = [
    {
        "quantity": "capacity",
        "sink":  "dummy",
        "overwrite": "true"
    }
]
dump(settings, (model_path / "modified_settings.toml").open("w"))
settings

{'time_framework': [2020, 2025, 2030, 2035, 2040, 2045, 2050],
 'foresight': 5,
 'regions': ['USA'],
 'interest_rate': 0.1,
 'interpolation_mode': 'Active',
 'log_level': 'info',
 'equilibrium_variable': 'demand',
 'maximum_iterations': 100,
 'tolerance': 0.1,
 'tolerance_unmet_demand': -0.1,
 'carbon_budget_control': {'budget': []},
 'global_input_files': {'projections': '{path}/input/Projections.csv',
  'global_commodities': '{path}/input/GlobalCommodities.csv'},
 'sectors': {'residential': {'type': 'default',
   'priority': 1,
   'investment_production': 'share',
   'dispatch_production': 'share',
   'agents': '{path}/technodata/Agents.csv',
   'technodata': '{path}/technodata/residential/Technodata.csv',
   'commodities_in': '{path}/technodata/residential/CommIn.csv',
   'commodities_out': '{path}/technodata/residential/CommOut.csv',
   'existing_capacity': '{path}/technodata/residential/Existing.csv',
   'outputs': [{'quantity': 'capacity', 'sink': 'dummy', 'overwrite': 'true'}],


At this point, we can run the simulation using the commandline (`python3 -m muse
settings2.toml`). And of course, we can run it from this notebook:

In [16]:
from logging import getLogger
getLogger("muse").setLevel(100) # suppresses the log
mca = MCA.factory(model_path / "modified_settings.toml")
mca.run()
assert (Path("Results") / "Residential2020Capacity.dummy").exists()